In [1]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns 

In [7]:
import sqlite3
conn = sqlite3.connect('bcp.db') 
c=conn.cursor()

In [8]:
df = pd.read_sql_query("SELECT * from df", conn)

In [9]:
conn.close()

In [11]:
df.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,1,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,1,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,1,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [12]:
#splitting the data into independent (X) and dependent (Y) datasets 
X= df.iloc[:,2:31].values #this will tell us the feature that can be used to detect if the patient has cancer or not
Y= df.iloc[:,1].values #this dataset will tell if the patient has cancer or not

In [15]:
#splitting the data set into 75% training set and 25% testing test 
from sklearn.model_selection import train_test_split 
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.25, random_state=0)

In [16]:
#scaling the data (Feature Scaling)
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test= sc.fit_transform(X_test)

In [17]:
X_train  

array([[-0.65079907, -0.43057322, -0.68024847, ..., -0.69592933,
        -0.36433881,  0.32349851],
       [-0.82835341,  0.15226547, -0.82773762, ..., -1.29277423,
        -1.45036679,  0.62563098],
       [ 1.68277234,  2.18977235,  1.60009756, ...,  0.26255563,
         0.72504581, -0.51329768],
       ...,
       [-1.33114223, -0.22172269, -1.3242844 , ..., -0.78274313,
        -0.98806491, -0.69995543],
       [-1.25110186, -0.24600763, -1.28700242, ..., -1.36015587,
        -1.75887319, -1.56206114],
       [-0.74662205,  1.14066273, -0.72203706, ...,  0.47201917,
        -0.2860679 , -1.24094654]])

In [20]:
#creating a function for the models 
def models(X_train, Y_train):
    
    #logistic Regression
    from sklearn.linear_model import LogisticRegression
    log = LogisticRegression(random_state=0)
    log.fit(X_train, Y_train)
    
    #Decision Tree
    from sklearn.tree import DecisionTreeClassifier
    tree = DecisionTreeClassifier (criterion ='entropy', random_state=0)
    tree.fit(X_train, Y_train)
    
    #Random Forest Classifier
    from sklearn.ensemble import RandomForestClassifier
    forest = RandomForestClassifier(n_estimators =10, criterion='entropy', random_state=0)
    forest.fit(X_train, Y_train)
    
    #Printing models accuracy on the training data
    print('[0]Logistic Regressioon Training Accuracy:', log.score(X_train,Y_train))
    print('[1]Decision Tree Training Accuracy:', tree.score(X_train,Y_train))
    print('[2]Random Forest Classifier Training Accuracy:', forest.score(X_train,Y_train))
    
    return log, tree, forest

In [21]:
#Getting all of the models
model = models(X_train, Y_train)

[0]Logistic Regressioon Training Accuracy: 0.9906103286384976
[1]Decision Tree Training Accuracy: 1.0
[2]Random Forest Classifier Training Accuracy: 0.9953051643192489


C:\Python37\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [24]:
#Testing model accuracy on test data on confusion matrix 
from sklearn.metrics import confusion_matrix
cm= confusion_matrix(Y_test, model[0].predict(X_test))
print(cm)   #TP=86,TN=50,FP=4,FN=3

[[86  4]
 [ 3 50]]


In [25]:
TP=cm[0][0]
TN=cm[1][1]
FP=cm[0][1]
FN=cm[1][0]

print('Testing Accuracy=',(TP+TN)/(TP+TN+FP+FN))

Testing Accuracy= 0.951048951048951


In [27]:
#To get the accuracy of all models we can create a loop
for i in range(len(model)):
    print('Model',i)
    cm= confusion_matrix(Y_test, model[i].predict(X_test))
    
    TP=cm[0][0]
    TN=cm[1][1]
    FP=cm[0][1]
    FN=cm[1][0]
    
    print(cm)
    print('Testing Accuracy=',(TP+TN)/(TP+TN+FP+FN))
    print()

Model 0
[[86  4]
 [ 3 50]]
Testing Accuracy= 0.951048951048951

Model 1
[[83  7]
 [ 2 51]]
Testing Accuracy= 0.9370629370629371

Model 2
[[87  3]
 [ 2 51]]
Testing Accuracy= 0.965034965034965



In [34]:
#another method to get matrix of the models:
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

for i in range(len(model)):
    print('Model',i)
    print (classification_report(Y_test, model[i].predict(X_test)))
    print (accuracy_score (Y_test, model[i].predict(X_test)))
    print()

Model 0
              precision    recall  f1-score   support

           0       0.97      0.96      0.96        90
           1       0.93      0.94      0.93        53

    accuracy                           0.95       143
   macro avg       0.95      0.95      0.95       143
weighted avg       0.95      0.95      0.95       143

0.951048951048951

Model 1
              precision    recall  f1-score   support

           0       0.98      0.92      0.95        90
           1       0.88      0.96      0.92        53

    accuracy                           0.94       143
   macro avg       0.93      0.94      0.93       143
weighted avg       0.94      0.94      0.94       143

0.9370629370629371

Model 2
              precision    recall  f1-score   support

           0       0.98      0.97      0.97        90
           1       0.94      0.96      0.95        53

    accuracy                           0.97       143
   macro avg       0.96      0.96      0.96       143
weighted av

In [35]:
#printing the prediction of the Random Forest Classifier model 
pred=model[2].predict(X_test)
print(pred)
print()
print(Y_test)

[1 0 0 0 0 0 0 0 0 0 1 0 0 1 1 1 0 1 1 1 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 0 0 1 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 0
 1 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0
 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1]

[1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 0 0 1 0 0 1 0 1 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 1 0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 1 0 0 1 0 1 1 1 0 0 1 0 1
 1 0 0 0 0 0 1 1 1 0 1 0 0 0 1 1 0 1 0 1 0 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 0
 1 1 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 0 0 0 1 1 0 0 0 1]


#### First list shows the result of the prediction and the second list shows the real result. The model had an accuracy of 96%. There are few instances where the model failed to predict correctly. So for example, 4% of the time, a model told a healthy person she had breast cancer and vice versa.
I am currently working to inprove the model's acccuracy. 